# Import Libraries and Set Constants


In [9]:
import os
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K 
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from efficientnet.tfkeras import EfficientNetB0

# Set TensorFlow version
print('TensorFlow version: ', tf.__version__)

# Set dataset paths
dataset_path = 'D:\Github\A-6th\CVIP Project\Deepfake-Detection-system-using-CNN\DeepFake-Detect\VerySmallData'
tmp_debug_path = './tmp_debug'
checkpoint_filepath = './tmp_checkpoint'

# Create directories if they don't exist
os.makedirs(tmp_debug_path, exist_ok=True)
os.makedirs(checkpoint_filepath, exist_ok=True)

# Function to extract filename without extension
def get_filename_only(file_path):
    file_basename = os.path.basename(file_path) 
    filename_only = file_basename.split('.')[0]
    return filename_only

# Set parameters
input_size = 128
batch_size_num = 32
num_epochs = 20


TensorFlow version:  2.14.0


# Data Preparation and Augmentation

In [10]:
# Data generators
train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    directory= os.path.join(dataset_path, 'train'),
    target_size=(input_size, input_size),
    color_mode="rgb",
    class_mode="binary",
    batch_size=batch_size_num,
    shuffle=True,
    save_to_dir=tmp_debug_path
)

val_generator = val_datagen.flow_from_directory(
    directory= os.path.join(dataset_path, 'val'),
    target_size=(input_size, input_size),
    color_mode="rgb",
    class_mode="binary",
    batch_size=batch_size_num,
    shuffle=True,
    save_to_dir=tmp_debug_path
)

test_generator = test_datagen.flow_from_directory(
    directory=os.path.join(dataset_path, 'test'),
    classes=['real', 'fake'],
    target_size=(input_size, input_size),
    color_mode="rgb",
    class_mode=None,
    batch_size=1,
    shuffle=False
)


Found 16 images belonging to 2 classes.
Found 2 images belonging to 2 classes.


Found 4 images belonging to 2 classes.


# Model Construction and Training

In [11]:
# Base model
efficient_net = EfficientNetB0(
    weights='imagenet',
    input_shape=(input_size, input_size, 3),
    include_top=False,
    pooling='max'
)

# Model architecture
model = Sequential()
model.add(efficient_net)
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

# Compile model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Define callbacks for training
custom_callbacks = [
    EarlyStopping(
        monitor='val_loss',
        mode='min',
        patience=5,
        verbose=1
    ),
    ModelCheckpoint(
        filepath=os.path.join(checkpoint_filepath, 'best_model.h5'),
        monitor='val_loss',
        mode='min',
        verbose=1,
        save_best_only=True
    )
]

# Train model
history = model.fit_generator(
    train_generator,
    epochs=num_epochs,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator),
    callbacks=custom_callbacks,
)


C:\Users\SONIC LAPTOPS\AppData\Local\Temp\ipykernel_8860\3867817507.py:38: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/20
1/1 [==============================] - ETA: 0s - loss: 1.1376 - accuracy: 0.5625
Epoch 1: val_loss improved from inf to 1.25185, saving model to ./tmp_checkpoint\best_model.h5


c:\Users\SONIC LAPTOPS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 28s 28s/step - loss: 1.1376 - accuracy: 0.5625 - val_loss: 1.2519 - val_accuracy: 0.5000
Epoch 2/20
1/1 [==============================] - ETA: 0s - loss: 2.5945 - accuracy: 0.5000
Epoch 2: val_loss improved from 1.25185 to 0.51384, saving model to ./tmp_checkpoint\best_model.h5
1/1 [==============================] - 2s 2s/step - loss: 2.5945 - accuracy: 0.5000 - val_loss: 0.5138 - val_accuracy: 1.0000
Epoch 3/20
1/1 [==============================] - ETA: 0s - loss: 0.1524 - accuracy: 0.9375
Epoch 3: val_loss did not improve from 0.51384
1/1 [==============================] - 2s 2s/step - loss: 0.1524 - accuracy: 0.9375 - val_loss: 1.0624 - val_accuracy: 0.5000
Epoch 4/20
1/1 [==============================] - ETA: 0s - loss: 1.0269 - accuracy: 0.7500
Epoch 4: val_loss did not improve from 0.51384
1/1 [==============================] - 2s 2s/step - loss: 1.0269 - accuracy: 0.7500 - val_loss: 0.8490 - val_accuracy: 0.5000
Epoch 5/20
1/1 [=========

# Model Evaluation and Prediction

In [12]:
# Load best model
best_model = load_model(os.path.join(checkpoint_filepath, 'best_model.h5'))

# Generate predictions
test_generator.reset()
preds = best_model.predict(
    test_generator,
    verbose=1
)
test_results = pd.DataFrame({
    "Filename": test_generator.filenames,
    "Prediction": preds.flatten()
}) 

# Add 'Label' column
test_results['Label'] = test_results['Prediction'].apply(lambda x: 'Real' if x > 0.5 else 'Fake')

print('Result After prediction = ',test_results)

# Save the results to a CSV file
test_results.to_csv('predictions1.csv', index=False)

print('Results saved to predictions.csv')


4/4 [==============================] - 2s 32ms/step
Result After prediction =                       Filename  Prediction Label
0  real\abarnvbtwb-000-00.png    0.500375  Real
1  real\abarnvbtwb-002-00.png    0.940016  Real
2  fake\aagfhgtpmv-009-00.png    0.127975  Fake
3  fake\aapnvogymq-004-01.png    0.000825  Fake
Results saved to predictions.csv
